<a href="https://colab.research.google.com/github/danabuogi/Logistic-Regression-Credit-Scoring-ML/blob/main/CreditScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [22]:
dataset = pd.read_excel("/content/gdrive/MyDrive/CreditScoringData/a_Dataset_CreditScoring.xlsx")

In [23]:
dataset.shape

(3000, 30)

In [24]:
dataset = dataset.drop("ID", axis = 1)
dataset.shape

(3000, 29)

In [25]:
dataset.head()

,TARGET,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,TLCnt03,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1.0,3.0,3.0,0.0,4.0,0.0,5.0,117.0,27.0,0.0,...,3.0,0.9179,0.2083,2.0,3.0,7.0,0.2083,4.0,4.0,0.0000
1,1.0,15.0,9.0,0.0,3.0,1.0,3.0,14.0,14.0,0.0,...,1.0,0.8000,0.0000,0.0,0.0,0.0,1.0000,12.0,0.0,1.0000
2,1.0,0.0,0.0,0.0,1.0,5.0,1.0,354.0,7.0,0.0,...,5.0,0.3552,0.6538,0.0,1.0,1.0,0.7308,1.0,1.0,0.5263
3,1.0,8.0,5.0,0.0,6.0,1.0,10.0,16.0,4.0,0.0,...,3.0,0.9127,0.2500,1.0,1.0,1.0,0.7500,7.0,1.0,1.3333
4,1.0,3.0,1.0,0.0,9.0,0.0,8.0,130.0,52.0,0.0,...,1.0,1.2511,0.0000,0.0,1.0,4.0,0.1429,3.0,1.0,0.0000


In [26]:
dataset.isna().sum() # explore missing values in each feature

TARGET               0
DerogCnt             0
CollectCnt           0
BanruptcyInd         0
InqCnt06             0
InqTimeLast        188
InqFinanceCnt24      0
TLTimeFirst          0
TLTimeLast           0
TLCnt03              0
TLCnt12              0
TLCnt24              0
TLCnt                3
TLSum               40
TLMaxSum            40
TLSatCnt             4
TLDel60Cnt           0
TLBadCnt24           0
TL75UtilCnt         99
TL50UtilCnt         99
TLBalHCPct          41
TLSatPct             4
TLDel3060Cnt24       0
TLDel90Cnt24         0
TLDel60CntAll        0
TLOpenPct            3
TLBadDerogCnt        0
TLDel60Cnt24         0
TLOpen24Pct          3
dtype: int64

In [27]:
# filling missing values with the mean value of each variable
dataset = dataset.fillna(dataset.mean())

In [28]:
dataset.isna().sum()

TARGET             0
DerogCnt           0
CollectCnt         0
BanruptcyInd       0
InqCnt06           0
InqTimeLast        0
InqFinanceCnt24    0
TLTimeFirst        0
TLTimeLast         0
TLCnt03            0
TLCnt12            0
TLCnt24            0
TLCnt              0
TLSum              0
TLMaxSum           0
TLSatCnt           0
TLDel60Cnt         0
TLBadCnt24         0
TL75UtilCnt        0
TL50UtilCnt        0
TLBalHCPct         0
TLSatPct           0
TLDel3060Cnt24     0
TLDel90Cnt24       0
TLDel60CntAll      0
TLOpenPct          0
TLBadDerogCnt      0
TLDel60Cnt24       0
TLOpen24Pct        0
dtype: int64

In [29]:
# count of bad loans (1) and good loans (0)
dataset["TARGET"].value_counts()

0.0    2500
1.0     500
Name: TARGET, dtype: int64

In [30]:
dataset.groupby("TARGET").mean()

,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,TLCnt03,TLCnt12,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
TARGET,,,,,,,,,,,,,,,,,,,,,
0.0,1.3224,0.7664,0.1492,2.9424,3.174638,3.2896,173.002,11.650,0.2844,1.832,...,3.986711,0.628177,0.544963,0.6044,0.6624,2.2236,0.502376,1.180,0.8648,0.556867
1.0,1.9680,1.3100,0.1740,3.9380,2.775459,4.8820,155.672,12.992,0.2280,1.768,...,4.533870,0.748185,0.385173,1.3340,1.5760,4.0140,0.465127,2.554,2.0860,0.600978


In [31]:
y = dataset.iloc[:, 0].values
X = dataset.iloc[:, 1:28].values

In [32]:
# splitting data into training and test in 80:20 ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [33]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Risk Modelling

In [34]:
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

Analyze Model Performance

In [35]:
print(confusion_matrix(y_test, y_pred))

[[482  16]
 [ 87  15]]


In [36]:
print(accuracy_score(y_test, y_pred))

0.8283333333333334


In [37]:
predictions = classifier.predict_proba(X_test)
print(predictions)

[[0.04230096 0.95769904]
 [0.93683282 0.06316718]
 [0.70459693 0.29540307]
 ...
 [0.9724934  0.0275066 ]
 [0.44332485 0.55667515]
 [0.86810657 0.13189343]]


Writing Model Output File

In [38]:
df_prediction_prob = pd.DataFrame(predictions, columns = ["prob_0", "prob_1"])
df_prediction_target = pd.DataFrame(classifier.predict(X_test), columns = ["predicted_TARGET"])
df_test_dataset = pd.DataFrame(y_test, columns = ["Actual Outcome"])

dfx = pd.concat([df_test_dataset, df_prediction_prob, df_prediction_target], axis = 1)
dfx.to_csv("/content/gdrive/MyDrive/CreditScoringData/c01_Model_Prediction.csv", sep = ",", encoding = "UTF-8")
dfx.head()

,Actual Outcome,prob_0,prob_1,predicted_TARGET
0,1.0,0.042301,0.957699,1.0
1,0.0,0.936833,0.063167,0.0
2,0.0,0.704597,0.295403,0.0
3,0.0,0.908269,0.091731,0.0
4,0.0,0.866217,0.133783,0.0
